#### **Case Study: Facility Layout Optimization using a Genetic Algorithm (Quadratic Assignment Problem)**

#### **Background**

You are a planning engineer for a manufacturing company that is setting up a new factory. The factory has 8 predefined locations where 8 different departments must be placed. The cost of operating the factory is heavily influenced by the material handling cost, which depends on the **flow of materials** between departments and the **distances** between the locations they are assigned to. Your task is to find the optimal assignment of departments to locations that **minimizes the total material handling cost**.

#### **Problem Description**

**Given:**

1.  **A Distance Matrix (`distance_matrix`)**: An 8x8 matrix where `distance_matrix[i][j]` is the distance between location `i` and location `j`.
2.  **A Flow Matrix (`flow_matrix`)**: An 8x8 matrix where `flow_matrix[i][j]` is the amount of material flow from department `i` to department `j`.

**Decision Variables (Genes):**

  * A solution is a permutation (a sequence) of 8 unique numbers from 0 to 7. For example, the solution `[3, 0, 1, 7, 6, 5, 4, 2]` means Department 0 is assigned to Location 3, Department 1 to Location 0, and so on.

**Objective (to be minimized):**

  * Minimize the total cost, calculated as: $Cost = \sum_{i=0}^{N-1} \sum_{j=0}^{N-1} Flow(i, j) \times Distance(p_i, p_j)$

#### **Programming Requirements**

1.  **Review the `GeneticAlgorithmQAP` Class**: A pre-built class for solving the QAP is provided below. It handles fitness evaluation, selection, crossover, and mutation.

2.  **Configure and Run the GA**: In the main script section at the bottom, your task is to create an instance of the `GeneticAlgorithmQAP` class. You must provide the correct values for the following parameters:
    * `population_size`
    * `num_generations`
    * `mutation_rate`
    
    Set these to reasonable values (e.g., `population_size = 100`, `num_generations = 200`, `mutation_rate = 0.05`).

3.  **Run the Notebook**: Execute all cells to run the optimization and see the final layout plot. The autograding system will evaluate the `Best Assignment Cost` printed in the output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Pre-built Genetic Algorithm Class for QAP ---
class GeneticAlgorithmQAP:
    """
    A Genetic Algorithm to solve the Quadratic Assignment Problem.
    """
    def __init__(self, num_locations, population_size, num_generations, mutation_rate, flow_matrix, distance_matrix):
        self.num_locations = num_locations
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.flow_matrix = flow_matrix
        self.distance_matrix = distance_matrix

    def calculate_qap_cost(self, assignment):
        """Calculates the total cost for a given department-to-location assignment."""
        total_cost = 0
        for i in range(self.num_locations):
            for j in range(self.num_locations):
                if i != j:
                    location_i = assignment[i]
                    location_j = assignment[j]
                    flow = self.flow_matrix[i, j]
                    distance = self.distance_matrix[location_i, location_j]
                    total_cost += flow * distance
        return total_cost

    def evaluate_fitness(self, population):
        """Evaluates fitness. Fitness is the inverse of cost for minimization."""
        costs = np.array([self.calculate_qap_cost(ind) for ind in population])
        return 1 / (costs + 1e-6)

    def generate_initial_population(self):
        return np.array([np.random.permutation(self.num_locations) for _ in range(self.population_size)])

    def selection_tournament(self, population, fitness, tournament_size=3):
        selected_indices = np.random.choice(len(population), tournament_size, replace=False)
        best_index = selected_indices[np.argmax(fitness[selected_indices])]
        return population[best_index]

    def crossover_ox(self, parent1, parent2):
        size = len(parent1)
        child = -np.ones(size, dtype=int)
        start, end = sorted(np.random.choice(range(size), 2, replace=False))
        child[start:end+1] = parent1[start:end+1]
        p2_ptr = 0
        for i in range(size):
            if child[i] == -1:
                while parent2[p2_ptr] in child:
                    p2_ptr += 1
                child[i] = parent2[p2_ptr]
        return child

    def mutate_swap(self, individual):
        if np.random.rand() < self.mutation_rate:
            idx1, idx2 = np.random.choice(range(len(individual)), 2, replace=False)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
        return individual

    def run(self):
        population = self.generate_initial_population()
        best_solution_overall = None
        best_cost_overall = np.inf
        for gen in range(self.num_generations):
            fitness = self.evaluate_fitness(population)
            new_population = []
            for _ in range(self.population_size // 2):
                parent1 = self.selection_tournament(population, fitness)
                parent2 = self.selection_tournament(population, fitness)
                offspring1 = self.crossover_ox(parent1, parent2)
                offspring2 = self.crossover_ox(parent2, parent1)
                new_population.append(self.mutate_swap(offspring1))
                new_population.append(self.mutate_swap(offspring2))
            population = np.array(new_population)
            current_fitness = self.evaluate_fitness(population)
            best_gen_idx = np.argmax(current_fitness)
            best_gen_cost = self.calculate_qap_cost(population[best_gen_idx])
            if best_gen_cost < best_cost_overall:
                best_cost_overall = best_gen_cost
                best_solution_overall = population[best_gen_idx]
            if (gen + 1) % 50 == 0:
                print(f"Generation {gen + 1}: Best Cost = {best_cost_overall:.2f}")
        print("\nOptimization Finished!")
        print(f"Best Assignment Cost: {best_cost_overall:.2f}")
        print(f"Best Assignment (Dept -> Loc): {best_solution_overall}")
        return best_solution_overall

def plot_layout(assignment, locations, dept_names):
    plt.figure(figsize=(8, 8))
    locs = np.array(locations)
    plt.scatter(locs[:, 0], locs[:, 1], s=500, c='skyblue', ec='black')
    for i, loc in enumerate(locs):
        assigned_dept_idx = np.where(assignment == i)[0][0]
        plt.text(loc[0], loc[1], f"Loc {i}\n({dept_names[assigned_dept_idx]})", ha='center', va='center', fontsize=10, weight='bold')
    plt.title("Optimal Facility Layout", fontsize=16)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

# --- Main Script ---
if __name__ == '__main__':
    NUM_DEPTS = 8
    LOCATIONS = [[0, 0], [0, 10], [0, 20], [10, 0], [10, 10], [10, 20], [20, 0], [20, 10]]
    DEPT_NAMES = [f"Dept {i}" for i in range(NUM_DEPTS)]

    dist_matrix = np.zeros((NUM_DEPTS, NUM_DEPTS))
    for i in range(NUM_DEPTS):
        for j in range(i, NUM_DEPTS):
            dist = np.linalg.norm(np.array(LOCATIONS[i]) - np.array(LOCATIONS[j]))
            dist_matrix[i, j] = dist_matrix[j, i] = dist

    flow_matrix = np.array([
        [0, 5, 2, 4, 1, 0, 0, 6],
        [5, 0, 3, 0, 2, 2, 0, 0],
        [2, 3, 0, 0, 0, 0, 5, 0],
        [4, 0, 0, 0, 5, 2, 0, 1],
        [1, 2, 0, 5, 0, 10, 0, 0],
        [0, 2, 0, 2, 10, 0, 5, 0],
        [0, 0, 5, 0, 0, 5, 0, 10],
        [6, 0, 0, 1, 0, 0, 10, 0]
    ])

    # --- YOUR TASK: Configure the GA Parameters ---
    # Create an instance of the GeneticAlgorithmQAP class below.
    # You need to provide integer/float values for population_size, num_generations, and mutation_rate.
    ga_qap = GeneticAlgorithmQAP(
        num_locations=NUM_DEPTS,
        population_size=  # YOUR CODE HERE: e.g., 100
        num_generations=  # YOUR CODE HERE: e.g., 200
        mutation_rate=    # YOUR CODE HERE: e.g., 0.05
        flow_matrix=flow_matrix,
        distance_matrix=dist_matrix
    )

    # Run optimization and plot results
    best_layout = ga_qap.run()
    plot_layout(best_layout, LOCATIONS, DEPT_NAMES)